In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests

### Structure matrice demandée ###

In [5]:
# Définir la plage de dates
date_range = pd.date_range(start="2024-01-01", end="2024-12-31", freq="D")

# Créer un DataFrame
df_vacances = pd.DataFrame({"Date": date_range})

# Ajouter des colonnes pour chaque région ou pays
regions = ["fr_paca", "fr_nord", "all_saxe"]
for region in regions:
    df_vacances[f"vac_{region}_début"] = False
    df_vacances[f"vac_{region}"] = False
    df_vacances[f"vac_{region}_fin"] = False

# Exemple de périodes de vacances
vacances = {
    "fr_paca": [("2024-02-10", "2024-02-25"), ("2024-07-01", "2024-08-31")],
    "fr_nord": [("2024-02-17", "2024-03-04"), ("2024-07-15", "2024-08-31")],
    "all_saxe": [("2024-02-12", "2024-02-23"), ("2024-07-10", "2024-08-20")]
}

# Remplir les colonnes en fonction des périodes
for region, periods in vacances.items():
    for start, end in periods:
        start_date = pd.to_datetime(start)
        end_date = pd.to_datetime(end)

        # Marquer le début, les vacances, et la fin
        df_vacances.loc[df_vacances["Date"] == start_date, f"vac_{region}_début"] = True
        df_vacances.loc[(df_vacances["Date"] >= start_date) & (df_vacances["Date"] <= end_date), f"vac_{region}"] = True
        df_vacances.loc[df_vacances["Date"] == end_date, f"vac_{region}_fin"] = True

# # Exporter le DataFrame si nécessaire
# df_vacances.to_excel("vacances_par_regions.xlsx", index=False)
df_vacances.head()

,Date,vac_fr_paca_début,vac_fr_paca,vac_fr_paca_fin,vac_fr_nord_début,vac_fr_nord,vac_fr_nord_fin,vac_all_saxe_début,vac_all_saxe,vac_all_saxe_fin
0,2024-01-01,False,False,False,False,False,False,False,False,False
1,2024-01-02,False,False,False,False,False,False,False,False,False
2,2024-01-03,False,False,False,False,False,False,False,False,False
3,2024-01-04,False,False,False,False,False,False,False,False,False
4,2024-01-05,False,False,False,False,False,False,False,False,False


### Jours Fériés France ###

In [14]:
# Définir les années et le pays
annees = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
pays = "FR"

# Initialiser une liste pour stocker les données de chaque année
fr_holidays = []

# Boucle pour récupérer les jours fériés pour chaque année
for annee in annees:
    url = f"https://date.nager.at/api/v3/publicholidays/{annee}/{pays}"
    response = requests.get(url)
    
    if response.status_code == 200:
        fr_holidays_data = response.json()  # Convertir la réponse en JSON
        fr_holidays.extend(fr_holidays_data)  # Ajouter les jours fériés à la liste
    else:
        print(f"Erreur lors de la récupération pour {annee}: {response.status_code}")

# Transformer en DataFrame
df_fr_holidays = pd.DataFrame(fr_holidays)

# Renommer les colonnes pour plus de clarté
df_fr_holidays.rename(columns={
    'date': 'Date',
    'localName': 'Nom Local',
    'name': 'Nom Anglais',
    'countryCode': 'Code Pays',
    'fixed': 'Fixe',
    'global': 'Global',
    'types': 'Type'
}, inplace=True)

# Convertir la colonne Date en type datetime
df_fr_holidays['Date'] = pd.to_datetime(df_fr_holidays['Date'])

# Trier le DataFrame par date
df_fr_holidays.sort_values(by='Date', inplace=True)

# Réinitialiser l'index
df_fr_holidays.reset_index(drop=True, inplace=True)

# # Exporter en Excel si nécessaire
# df_fr_holidays.to_excel("jours_feries_france_2019_2026.xlsx", index=False)
df_fr_holidays.head()

,Date,Nom Local,Nom Anglais,Code Pays,Fixe,Global,counties,launchYear,Type
0,2019-01-01,Jour de l'an,New Year's Day,FR,False,True,None,None,[Public]
1,2019-04-22,Lundi de Pâques,Easter Monday,FR,False,True,None,None,[Public]
2,2019-05-01,Fête du Travail,Labour Day,FR,False,True,None,None,[Public]
3,2019-05-08,Victoire 1945,Victory in Europe Day,FR,False,True,None,None,[Public]
4,2019-05-30,Ascension,Ascension Day,FR,False,True,None,None,[Public]


### Jours Fériés Europe ###

In [6]:
# Définir les années et les pays
annees = list(range(2019, 2027))  # De 2019 à 2026 inclus
pays_list = ["FR", "DE", "IT", "BE", "ES", "CH"]  # France, Allemagne, Italie, Belgique, Espagne, Suisse

# Initialiser une liste pour stocker les données
all_holidays = []

# Boucle pour récupérer les jours fériés pour chaque pays et chaque année
for pays in pays_list:
    for annee in annees:
        url = f"https://date.nager.at/api/v3/publicholidays/{annee}/{pays}"
        response = requests.get(url)
        
        if response.status_code == 200:
            holidays_data = response.json()
            
            # Ajouter une colonne pour identifier le pays
            for holiday in holidays_data:
                holiday["countryCode"] = pays
            
            # Ajouter les jours fériés à la liste globale
            all_holidays.extend(holidays_data)
        else:
            print(f"Erreur lors de la récupération pour {pays} en {annee}: {response.status_code}")

# Transformer en DataFrame
df_all_holidays = pd.DataFrame(all_holidays)

# Renommer les colonnes pour plus de clarté
df_all_holidays.rename(columns={
    'date': 'Date',
    'localName': 'Nom Local',
    'name': 'Nom Anglais',
    'countryCode': 'Code Pays',
    'fixed': 'Fixe',
    'global': 'Global',
    'types': 'Type'
}, inplace=True)

# Convertir la colonne Date en type datetime
df_all_holidays['Date'] = pd.to_datetime(df_all_holidays['Date'])

# Trier le DataFrame par date et pays
df_all_holidays.sort_values(by=['Date', 'Code Pays'], inplace=True)

# Réinitialiser l'index
df_all_holidays.reset_index(drop=True, inplace=True)

# # Exporter en Excel si nécessaire
# df_all_holidays.to_excel("jours_feries_europe_2019_2026.xlsx", index=False)
df_all_holidays.head()

,Date,Nom Local,Nom Anglais,Code Pays,Fixe,Global,counties,launchYear,Type
0,2019-01-01,Nieuwjaar,New Year's Day,BE,False,True,None,None,[Public]
1,2019-01-01,Neujahr,New Year's Day,CH,False,True,None,None,[Public]
2,2019-01-01,Neujahr,New Year's Day,DE,False,True,None,None,[Public]
3,2019-01-01,Año Nuevo,New Year's Day,ES,False,True,None,None,[Public]
4,2019-01-01,Jour de l'an,New Year's Day,FR,False,True,None,None,[Public]


### Vacances scolaires France 3 Zones ###

In [15]:
# Paramètres pour OpenHolidays API
country = "FR"  # Code pays (France)
language = "FR"  # Langue pour les noms
years = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]  # Années à récupérer
subdivisions = ["FR-ZA", "FR-ZB", "FR-ZC"]  # Zones

# Initialiser une liste pour stocker les résultats
all_fr_vacations = []

# Boucle sur les années et les subdivisions
for year in years:
    for subdivision in subdivisions:
        # Définir les dates de début et de fin
        valid_from = f"{year}-01-01"
        valid_to = f"{year}-12-31"

        # Construire l'URL pour l'année et la subdivision
        url = f"https://openholidaysapi.org/SchoolHolidays?countryIsoCode={country}&subdivisionCode={subdivision}&languageIsoCode={language}&validFrom={valid_from}&validTo={valid_to}"

        # Effectuer la requête
        response = requests.get(url)

        # Vérifier la réponse
        if response.status_code == 200:
            school_fr_vacations = response.json()

            # Ajouter une colonne pour l'année et la subdivision
            for vacation in school_fr_vacations:
                vacation["year"] = year
                vacation["subdivision"] = subdivision

            # Ajouter les données à la liste globale
            all_fr_vacations.extend(school_fr_vacations)
        else:
            print(f"Erreur pour {year} - {subdivision} : {response.status_code}")

# Transformer la liste complète en DataFrame
df_all_fr_vacations = pd.DataFrame(all_fr_vacations)

# # Exporter en Excel si nécessaire
# df_all_fr_vacations.to_excel("vacances_scolaires_France_2019_2026_Zones_A_B_C.xlsx", index=False)
df_all_fr_vacations.head()

,id,startDate,endDate,type,name,regionalScope,temporalScope,nationwide,subdivisions,year,subdivision
0,5d6383d7-06e5-4117-93b4-9ab8031406dc,2019-12-22,2020-01-05,School,"[{'language': 'FR', 'text': 'Vacances de Noël'}]",Regional,FullDay,False,"[{'code': 'FR-NA', 'shortName': 'NA'}, {'code'...",2019,FR-ZA
1,5d6383d7-06e5-4117-93b4-9ab8031406dc,2019-12-22,2020-01-05,School,"[{'language': 'FR', 'text': 'Vacances de Noël'}]",Regional,FullDay,False,"[{'code': 'FR-NA', 'shortName': 'NA'}, {'code'...",2019,FR-ZB
2,5d6383d7-06e5-4117-93b4-9ab8031406dc,2019-12-22,2020-01-05,School,"[{'language': 'FR', 'text': 'Vacances de Noël'}]",Regional,FullDay,False,"[{'code': 'FR-NA', 'shortName': 'NA'}, {'code'...",2019,FR-ZC
3,5d6383d7-06e5-4117-93b4-9ab8031406dc,2019-12-22,2020-01-05,School,"[{'language': 'FR', 'text': 'Vacances de Noël'}]",Regional,FullDay,False,"[{'code': 'FR-NA', 'shortName': 'NA'}, {'code'...",2020,FR-ZA
4,428fa6f9-f4cf-48f1-8c1a-8d615e49ba78,2020-02-23,2020-03-08,School,"[{'language': 'FR', 'text': 'Vacances d'hiver'}]",Regional,FullDay,False,"[{'code': 'FR-NA', 'shortName': 'NA'}, {'code'...",2020,FR-ZA


### Vacances scolaires Europe ###

In [10]:
# Paramètres pour OpenHolidays API
countries = ["FR", "IT", "DE", "CH", "BE", "ES"]  # Pays : France, Italie, Allemagne, Suisse, Belgique, Espagne
language = "FR"  # Langue pour les noms 
years = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]  # Années à récupérer

# Initialiser une liste pour stocker les résultats
all_vacations = []

# Boucle sur les années et les pays
for country in countries:
    for year in years:
        # Définir les dates de début et de fin
        valid_from = f"{year}-01-01"
        valid_to = f"{year}-12-31"

        # Construire l'URL pour l'année et le pays
        url = f"https://openholidaysapi.org/SchoolHolidays?countryIsoCode={country}&languageIsoCode={language}&validFrom={valid_from}&validTo={valid_to}"

        # Effectuer la requête
        response = requests.get(url)

        # Vérifier la réponse
        if response.status_code == 200:
            school_vacations = response.json()

            # Ajouter une colonne pour l'année et le pays
            for vacation in school_vacations:
                vacation["year"] = year
                vacation["country"] = country

            # Ajouter les données à la liste globale
            all_vacations.extend(school_vacations)
        else:
            print(f"Erreur pour {year} - {country} : {response.status_code}")

# Transformer la liste complète en DataFrame
df_all_vacations = pd.DataFrame(all_vacations)

# # Exporter en Excel si nécessaire
# df_all_vacations.to_excel("vacances_scolaires_Europe_2019_2026.xlsx", index=False)
df_all_vacations.head()

,id,startDate,endDate,type,name,regionalScope,temporalScope,nationwide,subdivisions,year,country,comment
0,5d6383d7-06e5-4117-93b4-9ab8031406dc,2019-12-22,2020-01-05,School,"[{'language': 'FR', 'text': 'Vacances de Noël'}]",Regional,FullDay,False,"[{'code': 'FR-NA', 'shortName': 'NA'}, {'code'...",2019,FR,NaN
1,5d6383d7-06e5-4117-93b4-9ab8031406dc,2019-12-22,2020-01-05,School,"[{'language': 'FR', 'text': 'Vacances de Noël'}]",Regional,FullDay,False,"[{'code': 'FR-NA', 'shortName': 'NA'}, {'code'...",2020,FR,NaN
2,a113cd51-afb7-43bb-96bd-bf0931c195b1,2020-02-09,2020-02-23,School,"[{'language': 'FR', 'text': 'Vacances d'hiver'}]",Regional,FullDay,False,"[{'code': 'FR-IF', 'shortName': 'IF'}, {'code'...",2020,FR,NaN
3,d64618fa-43bc-4a99-ad6b-d67bf18df25e,2020-02-16,2020-03-01,School,"[{'language': 'FR', 'text': 'Vacances d'hiver'}]",Regional,FullDay,False,"[{'code': 'FR-ZB', 'shortName': 'ZB'}, {'code'...",2020,FR,NaN
4,428fa6f9-f4cf-48f1-8c1a-8d615e49ba78,2020-02-23,2020-03-08,School,"[{'language': 'FR', 'text': 'Vacances d'hiver'}]",Regional,FullDay,False,"[{'code': 'FR-NA', 'shortName': 'NA'}, {'code'...",2020,FR,NaN


### Vacances scolaires Europe sous forme de matrice ###

In [11]:
# Paramètres pour OpenHolidays API
countries = ["FR", "IT", "DE", "CH", "BE", "ES"]  # France, Italie, Allemagne, Suisse, Belgique, Espagne
language = "FR"  # Langue pour les noms
years = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]  # Années à récupérer

# Définir la période couverte
date_range = pd.date_range(start="2019-01-01", end="2026-12-31", freq="D")

# Construire la base (matrice de dates)
date_matrix = pd.DataFrame({"Date": date_range})

# Ajouter des colonnes pour chaque pays
for country in countries:
    date_matrix[f"vac_{country.lower()}_début"] = False
    date_matrix[f"vac_{country.lower()}"] = False
    date_matrix[f"vac_{country.lower()}_fin"] = False

# Récupérer les données de vacances depuis l'API
for country in countries:
    all_vacations = []

    for year in years:
        # Définir les dates de début et de fin
        valid_from = f"{year}-01-01"
        valid_to = f"{year}-12-31"

        # Construire l'URL
        url = f"https://openholidaysapi.org/SchoolHolidays?countryIsoCode={country}&languageIsoCode={language}&validFrom={valid_from}&validTo={valid_to}"

        # Effectuer la requête
        response = requests.get(url)

        if response.status_code == 200:
            school_vacations = response.json()

            for vacation in school_vacations:
                start_date = pd.to_datetime(vacation["startDate"])
                end_date = pd.to_datetime(vacation["endDate"])
                all_vacations.append({"start": start_date, "end": end_date})
        else:
            print(f"Erreur pour {year} - {country} : {response.status_code}")

    # Marquer les dates dans la matrice
    for vacation in all_vacations:
        start_date = vacation["start"]
        end_date = vacation["end"]

        # Début des vacances
        date_matrix.loc[date_matrix["Date"] == start_date, f"vac_{country.lower()}_début"] = True

        # Période des vacances
        date_matrix.loc[(date_matrix["Date"] >= start_date) & (date_matrix["Date"] <= end_date), f"vac_{country.lower()}"] = True

        # Fin des vacances
        date_matrix.loc[date_matrix["Date"] == end_date, f"vac_{country.lower()}_fin"] = True

# # Exporter en Excel
# date_matrix.to_excel("matrice_vacances_scolaires_europe_2019_2026.xlsx", index=False)
date_matrix.head()

,Date,vac_fr_début,vac_fr,vac_fr_fin,vac_it_début,vac_it,vac_it_fin,vac_de_début,vac_de,vac_de_fin,vac_ch_début,vac_ch,vac_ch_fin,vac_be_début,vac_be,vac_be_fin,vac_es_début,vac_es,vac_es_fin
0,2019-01-01,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,2019-01-02,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,2019-01-03,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,2019-01-04,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False
4,2019-01-05,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


### Test matrice vacances et fériés Europe ###

In [16]:
# Récupérer les jours fériés:
# Définir les années et les pays
annees = list(range(2019, 2027))  # De 2019 à 2026 inclus
pays_list = ["FR", "DE", "IT", "BE", "ES", "CH"]  # France, Allemagne, Italie, Belgique, Espagne, Suisse

# Initialiser une liste pour les jours fériés
all_holidays = []

# Boucle pour récupérer les jours fériés pour chaque pays et chaque année
for pays in pays_list:
    for annee in annees:
        url = f"https://date.nager.at/api/v3/publicholidays/{annee}/{pays}"
        response = requests.get(url)
        
        if response.status_code == 200:
            holidays_data = response.json()
            for holiday in holidays_data:
                all_holidays.append({
                    "Date": pd.to_datetime(holiday['date']),
                    "Pays": pays,
                    "Nom": holiday['localName']
                })
        else:
            print(f"Erreur pour {pays} en {annee}: {response.status_code}")

# Transformer en DataFrame
df_holidays = pd.DataFrame(all_holidays)

# Récupérer les vacances scolaires:
# Paramètres pour OpenHolidays API
countries = ["FR", "IT", "DE", "CH", "BE", "ES"]  # France, Italie, Allemagne, Suisse, Belgique, Espagne
language = "FR"  # Langue pour les noms
years = list(range(2019, 2027))  # Années 2019 à 2026

# Initialiser une liste pour stocker les vacances scolaires
all_vacations = []

for country in countries:
    for year in years:
        valid_from = f"{year}-01-01"
        valid_to = f"{year}-12-31"
        url = f"https://openholidaysapi.org/SchoolHolidays?countryIsoCode={country}&languageIsoCode={language}&validFrom={valid_from}&validTo={valid_to}"
        response = requests.get(url)
        
        if response.status_code == 200:
            vacations_data = response.json()
            for vacation in vacations_data:
                all_vacations.append({
                    "Pays": country,
                    "Début": pd.to_datetime(vacation['startDate']),
                    "Fin": pd.to_datetime(vacation['endDate']),
                    "Nom": vacation.get("name", "Vacances Scolaires")
                })
        else:
            print(f"Erreur pour {country} en {year}: {response.status_code}")

# Transformer les vacances en DataFrame
df_vacations = pd.DataFrame(all_vacations)

# Fusionner les jours fériés et les vacances scolaires:
# Créer une matrice de dates et pays (2019 à 2026)
date_range = pd.date_range(start="2019-01-01", end="2026-12-31", freq="D")
test_complete_date_matrix = pd.DataFrame([(date, country) for date in date_range for country in countries], columns=["Date", "Pays"])

# Ajouter des colonnes pour les vacances et jours fériés
test_complete_date_matrix["vacances_début"] = False
test_complete_date_matrix["vacances"] = False
test_complete_date_matrix["vacances_fin"] = False
test_complete_date_matrix["jour_férié"] = False
test_complete_date_matrix["nom_férié"] = ""

# Marquer les jours fériés dans la matrice
for _, row in df_holidays.iterrows():
    mask = (test_complete_date_matrix["Date"] == row["Date"]) & (test_complete_date_matrix["Pays"] == row["Pays"])
    test_complete_date_matrix.loc[mask, "jour_férié"] = True
    test_complete_date_matrix.loc[mask, "nom_férié"] = row["Nom"]

# Marquer les vacances scolaires dans la matrice
for _, row in df_vacations.iterrows():
    mask_debut = (test_complete_date_matrix["Date"] == row["Début"]) & (test_complete_date_matrix["Pays"] == row["Pays"])
    mask_fin = (test_complete_date_matrix["Date"] == row["Fin"]) & (test_complete_date_matrix["Pays"] == row["Pays"])
    mask_vacances = (test_complete_date_matrix["Date"] >= row["Début"]) & (test_complete_date_matrix["Date"] <= row["Fin"]) & (test_complete_date_matrix["Pays"] == row["Pays"])
    test_complete_date_matrix.loc[mask_debut, "vacances_début"] = True
    test_complete_date_matrix.loc[mask_vacances, "vacances"] = True
    test_complete_date_matrix.loc[mask_fin, "vacances_fin"] = True

# # Exporter les données en Excel
# test_complete_date_matrix.to_excel("test_matrice_jours_feries_et_vacances_europe_2019_2026.xlsx", index=False)
test_complete_date_matrix.head()

,Date,Pays,vacances_début,vacances,vacances_fin,jour_férié,nom_férié
0,2019-01-01,FR,False,False,False,True,Jour de l'an
1,2019-01-01,IT,False,False,False,True,Capodanno
2,2019-01-01,DE,False,True,False,True,Neujahr
3,2019-01-01,CH,False,False,False,True,Neujahr
4,2019-01-01,BE,False,False,False,True,Nieuwjaar


### Matrice vacances et fériés Europe ###

In [17]:
# Récupérer les jours fériés
annees = list(range(2019, 2027))  # De 2019 à 2026 inclus
pays_list = ["FR", "DE", "IT", "BE", "ES", "CH"]  # Pays à inclure

# Initialiser une liste pour les jours fériés
all_holidays = []

for pays in pays_list:
    for annee in annees:
        url = f"https://date.nager.at/api/v3/publicholidays/{annee}/{pays}"
        response = requests.get(url)
        
        if response.status_code == 200:
            holidays_data = response.json()
            for holiday in holidays_data:
                all_holidays.append({
                    "Pays": pays,
                    "Date": pd.to_datetime(holiday["date"]),
                    "Nom": holiday["localName"],
                    "Type": "jour_férié"
                })
        else:
            print(f"Erreur pour {pays} en {annee}: {response.status_code}")

# Transformer les jours fériés en DataFrame
df_holidays = pd.DataFrame(all_holidays)

# Récupérer les vacances scolaires
countries = ["FR", "IT", "DE", "CH", "BE", "ES"]
language = "FR"
years = list(range(2019, 2027))
all_vacations = []

for country in countries:
    for year in years:
        valid_from = f"{year}-01-01"
        valid_to = f"{year}-12-31"
        url = f"https://openholidaysapi.org/SchoolHolidays?countryIsoCode={country}&languageIsoCode={language}&validFrom={valid_from}&validTo={valid_to}"
        response = requests.get(url)
        
        if response.status_code == 200:
            vacations_data = response.json()
            for vacation in vacations_data:
                all_vacations.append({
                    "Pays": country,
                    "Début": pd.to_datetime(vacation["startDate"]),
                    "Fin": pd.to_datetime(vacation["endDate"]),
                    "Nom": vacation.get("name", "Vacances Scolaires"),
                    "Type": "vacances"
                })
        else:
            print(f"Erreur pour {country} en {year}: {response.status_code}")

# Transformer les vacances en DataFrame
df_vacations = pd.DataFrame(all_vacations)

# Créer la matrice finale:
# Définir la plage de dates
date_range = pd.date_range(start="2019-01-01", end="2026-12-31", freq="D")
complete_date_matrix = pd.DataFrame({"Date": date_range})

# Ajouter des colonnes pour chaque pays (jours fériés et vacances)
for country in pays_list:
    complete_date_matrix[f"vac_{country.lower()}_début"] = False
    complete_date_matrix[f"vac_{country.lower()}"] = False
    complete_date_matrix[f"vac_{country.lower()}_fin"] = False
    complete_date_matrix[f"ferie_{country.lower()}"] = False

# Remplir la matrice avec les vacances scolaires
for _, row in df_vacations.iterrows():
    country = row["Pays"].lower()
    mask_debut = complete_date_matrix["Date"] == row["Début"]
    mask_vacances = (complete_date_matrix["Date"] >= row["Début"]) & (complete_date_matrix["Date"] <= row["Fin"])
    mask_fin = complete_date_matrix["Date"] == row["Fin"]

    complete_date_matrix.loc[mask_debut, f"vac_{country}_début"] = True
    complete_date_matrix.loc[mask_vacances, f"vac_{country}"] = True
    complete_date_matrix.loc[mask_fin, f"vac_{country}_fin"] = True

# Remplir la matrice avec les jours fériés
for _, row in df_holidays.iterrows():
    country = row["Pays"].lower()
    mask_ferie = complete_date_matrix["Date"] == row["Date"]
    complete_date_matrix.loc[mask_ferie, f"ferie_{country}"] = True

# # Exporter le fichier
# complete_date_matrix.to_excel("matrice_vacances_feries_europe_2019_2026.xlsx", index=False)
complete_date_matrix.head()

,Date,vac_fr_début,vac_fr,vac_fr_fin,ferie_fr,vac_de_début,vac_de,vac_de_fin,ferie_de,vac_it_début,...,vac_be_fin,ferie_be,vac_es_début,vac_es,vac_es_fin,ferie_es,vac_ch_début,vac_ch,vac_ch_fin,ferie_ch
0,2019-01-01,False,False,False,True,False,True,False,True,False,...,False,True,False,False,False,True,False,False,False,True
1,2019-01-02,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,2019-01-03,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2019-01-04,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2019-01-05,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Test vacances et fériés Europe avec régions France ###

In [18]:
# Récupérer les vacances régionales françaises
country = "FR"
language = "FR"
years = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
subdivisions = ["FR-ZA", "FR-ZB", "FR-ZC"]
all_vacations = []

for year in years:
    for subdivision in subdivisions:
        valid_from = f"{year}-01-01"
        valid_to = f"{year}-12-31"
        url = f"https://openholidaysapi.org/SchoolHolidays?countryIsoCode={country}&subdivisionCode={subdivision}&languageIsoCode={language}&validFrom={valid_from}&validTo={valid_to}"
        response = requests.get(url)

        if response.status_code == 200:
            school_vacations = response.json()
            for vacation in school_vacations:
                all_vacations.append({
                    "Début": pd.to_datetime(vacation["startDate"]),
                    "Fin": pd.to_datetime(vacation["endDate"]),
                    "Region": subdivision,
                    "vacances": True
                })
        else:
            print(f"Erreur pour {year} - {subdivision} : {response.status_code}")

# Transformer les vacances en DataFrame
df_all_vacations = pd.DataFrame(all_vacations)

# Étendre les plages de dates pour ajouter une colonne `Date`
expanded_rows = []
for _, row in df_all_vacations.iterrows():
    start_date = row["Début"]
    end_date = row["Fin"]
    for single_date in pd.date_range(start=start_date, end=end_date):
        expanded_rows.append({
            "Date": single_date,
            "Region": row["Region"],
            "vacances": True
        })

# Créer un nouveau DataFrame avec les dates étendues
df_all_vacations_expanded = pd.DataFrame(expanded_rows)

# Récupérer les jours fériés
annees = list(range(2019, 2027))
pays_list = ["FR", "DE", "IT", "BE", "ES", "CH"]
all_holidays = []

for pays in pays_list:
    for annee in annees:
        url = f"https://date.nager.at/api/v3/publicholidays/{annee}/{pays}"
        response = requests.get(url)
        
        if response.status_code == 200:
            holidays_data = response.json()
            for holiday in holidays_data:
                all_holidays.append({
                    "Pays": pays,
                    "Date": pd.to_datetime(holiday["date"]),
                    "Nom": holiday["localName"],
                    "Type": "jour_férié"
                })
        else:
            print(f"Erreur pour {pays} en {annee}: {response.status_code}")

# Transformer les jours fériés en DataFrame
df_holidays = pd.DataFrame(all_holidays)

# Créer la matrice principale
date_range = pd.date_range(start="2019-01-01", end="2026-12-31", freq="D")
test_date_matrix = pd.DataFrame({"Date": date_range})

# Ajouter des colonnes pour chaque pays (jours fériés et vacances)
for country in pays_list:
    test_date_matrix[f"vac_{country.lower()}_début"] = False
    test_date_matrix[f"vac_{country.lower()}"] = False
    test_date_matrix[f"vac_{country.lower()}_fin"] = False
    test_date_matrix[f"ferie_{country.lower()}"] = False

# Remplir la matrice avec les vacances scolaires
for _, row in df_all_vacations_expanded.iterrows():
    region = row["Region"].lower()
    date = row["Date"]
    mask_date = test_date_matrix["Date"] == date

    # Ajouter la colonne de vacances pour les régions
    if f"vac_{region}" not in test_date_matrix.columns:
        test_date_matrix[f"vac_{region}"] = False

    test_date_matrix.loc[mask_date, f"vac_{region}"] = True

# Remplir la matrice avec les jours fériés
for _, row in df_holidays.iterrows():
    country = row["Pays"].lower()
    mask_ferie = test_date_matrix["Date"] == row["Date"]
    test_date_matrix.loc[mask_ferie, f"ferie_{country}"] = True

# Ajouter des colonnes `début` et `fin` pour chaque subdivision française
for region in subdivisions:
    col_vac = f"vac_{region.lower()}"
    col_debut = f"{col_vac}_début"
    col_fin = f"{col_vac}_fin"

    # Assurer que la colonne est booléenne
    if col_vac in test_date_matrix.columns:
        test_date_matrix[col_vac] = test_date_matrix[col_vac].astype(bool)
        
        # Identifier les débuts et fins
        test_date_matrix[col_debut] = test_date_matrix[col_vac] & ~test_date_matrix[col_vac].shift(1, fill_value=False).astype(bool)
        test_date_matrix[col_fin] = test_date_matrix[col_vac] & ~test_date_matrix[col_vac].shift(-1, fill_value=False).astype(bool)

# # Exporter la matrice finale
# test_date_matrix.to_excel("test_matrice_vacances_feries_europe_et_regions_fr_2019_2026.xlsx", index=False)
test_date_matrix.head()

,Date,vac_fr_début,vac_fr,vac_fr_fin,ferie_fr,vac_de_début,vac_de,vac_de_fin,ferie_de,vac_it_début,...,ferie_ch,vac_fr-za,vac_fr-zb,vac_fr-zc,vac_fr-za_début,vac_fr-za_fin,vac_fr-zb_début,vac_fr-zb_fin,vac_fr-zc_début,vac_fr-zc_fin
0,2019-01-01,False,False,False,True,False,False,False,True,False,...,True,False,False,False,False,False,False,False,False,False
1,2019-01-02,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,2019-01-03,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2019-01-04,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2019-01-05,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Test2 vacances et fériés Europe avec régions France ###

In [20]:
# Récupérer les vacances régionales françaises
country = "FR"
language = "FR"
years = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
subdivisions = ["FR-ZA", "FR-ZB", "FR-ZC"]
all_vacations = []

for year in years:
    for subdivision in subdivisions:
        valid_from = f"{year}-01-01"
        valid_to = f"{year}-12-31"
        url = f"https://openholidaysapi.org/SchoolHolidays?countryIsoCode={country}&subdivisionCode={subdivision}&languageIsoCode={language}&validFrom={valid_from}&validTo={valid_to}"
        response = requests.get(url)

        if response.status_code == 200:
            school_vacations = response.json()
            for vacation in school_vacations:
                all_vacations.append({
                    "Début": pd.to_datetime(vacation["startDate"]),
                    "Fin": pd.to_datetime(vacation["endDate"]),
                    "Region": subdivision,
                    "vacances": True
                })
        else:
            print(f"Erreur pour {year} - {subdivision} : {response.status_code}")

# Transformer les vacances en DataFrame
df_all_vacations = pd.DataFrame(all_vacations)

# Étendre les plages de dates pour ajouter une colonne `Date`
expanded_rows = []
for _, row in df_all_vacations.iterrows():
    start_date = row["Début"]
    end_date = row["Fin"]
    for single_date in pd.date_range(start=start_date, end=end_date):
        expanded_rows.append({
            "Date": single_date,
            "Region": row["Region"],
            "vacances": True
        })

# Créer un nouveau DataFrame avec les dates étendues
df_all_vacations_expanded = pd.DataFrame(expanded_rows)

# Récupérer les jours fériés
annees = list(range(2019, 2027))
pays_list = ["FR", "DE", "IT", "BE", "ES", "CH"]
all_holidays = []

for pays in pays_list:
    for annee in annees:
        url = f"https://date.nager.at/api/v3/publicholidays/{annee}/{pays}"
        response = requests.get(url)
        
        if response.status_code == 200:
            holidays_data = response.json()
            for holiday in holidays_data:
                all_holidays.append({
                    "Pays": pays,
                    "Date": pd.to_datetime(holiday["date"]),
                    "Nom": holiday["localName"],
                    "Type": "jour_férié"
                })
        else:
            print(f"Erreur pour {pays} en {annee}: {response.status_code}")

# Transformer les jours fériés en DataFrame
df_holidays = pd.DataFrame(all_holidays)

# Créer la matrice principale
date_range = pd.date_range(start="2019-01-01", end="2026-12-31", freq="D")
test_date_matrix2 = pd.DataFrame({"Date": date_range})

# Ajouter des colonnes pour chaque pays (jours fériés et vacances)
for country in pays_list:
    test_date_matrix2[f"vac_{country.lower()}_début"] = False
    test_date_matrix2[f"vac_{country.lower()}"] = False
    test_date_matrix2[f"vac_{country.lower()}_fin"] = False
    test_date_matrix2[f"ferie_{country.lower()}"] = False

# Remplir la matrice avec les vacances scolaires régionales
for _, row in df_all_vacations_expanded.iterrows():
    region = row["Region"].lower()
    date = row["Date"]
    mask_date = test_date_matrix2["Date"] == date

    # Ajouter la colonne de vacances pour les régions
    if f"vac_{region}" not in test_date_matrix2.columns:
        test_date_matrix2[f"vac_{region}"] = False

    test_date_matrix2.loc[mask_date, f"vac_{region}"] = True

# Supprimer les colonnes globales pour la France
test_date_matrix2.drop(columns=["vac_fr", "vac_fr_début", "vac_fr_fin"], errors="ignore", inplace=True)

# Fusionner les colonnes régionales pour créer les colonnes globales pour la France
test_date_matrix2["vac_fr"] = test_date_matrix2[[f"vac_{region.lower()}" for region in subdivisions]].any(axis=1)
test_date_matrix2["vac_fr_début"] = test_date_matrix2["vac_fr"] & ~test_date_matrix2["vac_fr"].shift(1, fill_value=False)
test_date_matrix2["vac_fr_fin"] = test_date_matrix2["vac_fr"] & ~test_date_matrix2["vac_fr"].shift(-1, fill_value=False)

# Remplir la matrice avec les jours fériés
for _, row in df_holidays.iterrows():
    country = row["Pays"].lower()
    mask_ferie = test_date_matrix2["Date"] == row["Date"]
    test_date_matrix2.loc[mask_ferie, f"ferie_{country}"] = True

# # Exporter la matrice finale
# test_date_matrix2.to_excel("test2_matrice_vacances_feries_europe_et_regions_fr_2019_2026.xlsx", index=False)
test_date_matrix2.head()

,Date,ferie_fr,vac_de_début,vac_de,vac_de_fin,ferie_de,vac_it_début,vac_it,vac_it_fin,ferie_it,...,vac_ch_début,vac_ch,vac_ch_fin,ferie_ch,vac_fr-za,vac_fr-zb,vac_fr-zc,vac_fr,vac_fr_début,vac_fr_fin
0,2019-01-01,True,False,False,False,True,False,False,False,True,...,False,False,False,True,False,False,False,False,False,False
1,2019-01-02,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,2019-01-03,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2019-01-04,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2019-01-05,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Vacances et fériés Europe avec régions ###

In [21]:
# Récupérer les subdivisions pour chaque pays
def get_subdivisions(country):
    """Récupère les subdivisions pour un pays donné depuis l'API OpenHolidays."""
    url = f"https://openholidaysapi.org/Subdivisions?countryIsoCode={country}"
    response = requests.get(url)
    if response.status_code == 200:
        subdivisions_data = response.json()
        return [sub["code"] for sub in subdivisions_data]
    else:
        print(f"Erreur lors de la récupération des subdivisions pour {country} : {response.status_code}")
        return []

# Liste des pays à inclure
countries = ["FR", "IT", "DE", "CH", "BE", "ES"]
language = "FR"
years = list(range(2019, 2027))

# Récupérer toutes les subdivisions
subdivisions_per_country = {}
for country in countries:
    subdivisions_per_country[country] = get_subdivisions(country)

# Récupérer les vacances scolaires pour chaque subdivision
all_vacations = []
for country in countries:
    subdivisions = subdivisions_per_country[country]
    for year in years:
        for subdivision in subdivisions:
            valid_from = f"{year}-01-01"
            valid_to = f"{year}-12-31"
            url = f"https://openholidaysapi.org/SchoolHolidays?countryIsoCode={country}&subdivisionCode={subdivision}&languageIsoCode={language}&validFrom={valid_from}&validTo={valid_to}"
            response = requests.get(url)
            if response.status_code == 200:
                vacations_data = response.json()
                for vacation in vacations_data:
                    all_vacations.append({
                        "Pays": country,
                        "Région": subdivision,
                        "Début": pd.to_datetime(vacation["startDate"]),
                        "Fin": pd.to_datetime(vacation["endDate"]),
                        "Nom": vacation.get("name", "Vacances Scolaires"),
                        "Type": "vacances"
                    })
            else:
                print(f"Erreur pour {country} - {subdivision} en {year}: {response.status_code}")

# Transformer les vacances en DataFrame
df_vacations = pd.DataFrame(all_vacations)

# Étendre les plages de dates pour chaque vacances
expanded_vacations = []
for _, row in df_vacations.iterrows():
    for single_date in pd.date_range(start=row["Début"], end=row["Fin"]):
        expanded_vacations.append({
            "Date": single_date,
            "Pays": row["Pays"],
            "Région": row["Région"],
            "Nom": row["Nom"],
            "Type": row["Type"]
        })
df_vacations_expanded = pd.DataFrame(expanded_vacations)

# Récupérer les jours fériés
all_holidays = []
for country in countries:
    for year in years:
        url = f"https://date.nager.at/api/v3/publicholidays/{year}/{country}"
        response = requests.get(url)
        if response.status_code == 200:
            holidays_data = response.json()
            for holiday in holidays_data:
                all_holidays.append({
                    "Date": pd.to_datetime(holiday["date"]),
                    "Pays": country,
                    "Nom": holiday["localName"],
                    "Type": "jour_férié"
                })
        else:
            print(f"Erreur pour {country} en {year}: {response.status_code}")

# Transformer les jours fériés en DataFrame
df_holidays = pd.DataFrame(all_holidays)

# Fusionner vacances et jours fériés
df_combined = pd.concat([df_vacations_expanded, df_holidays], ignore_index=True)

# Créer la matrice finale
date_range = pd.date_range(start="2019-01-01", end="2026-12-31", freq="D")
all_date_matrix = pd.DataFrame({"Date": date_range})

# Ajouter des colonnes pour chaque pays et subdivision
for country in countries:
    all_date_matrix[f"ferie_{country.lower()}"] = False
    for subdivision in subdivisions_per_country[country]:
        region_col = f"vac_{country.lower()}_{subdivision.lower()}"
        all_date_matrix[region_col] = False
        all_date_matrix[f"{region_col}_début"] = False
        all_date_matrix[f"{region_col}_fin"] = False

# Remplir la matrice avec les vacances
for _, row in df_vacations_expanded.iterrows():
    country = row["Pays"].lower()
    region = row["Région"].lower()
    date = row["Date"]
    region_col = f"vac_{country}_{region}"
    
    mask_date = all_date_matrix["Date"] == date
    all_date_matrix.loc[mask_date, region_col] = True

# Identifier les débuts et fins des vacances par région
for country in countries:
    for subdivision in subdivisions_per_country[country]:
        region_col = f"vac_{country.lower()}_{subdivision.lower()}"
        debut_col = f"{region_col}_début"
        fin_col = f"{region_col}_fin"

        all_date_matrix[debut_col] = all_date_matrix[region_col] & ~all_date_matrix[region_col].shift(1, fill_value=False)
        all_date_matrix[fin_col] = all_date_matrix[region_col] & ~all_date_matrix[region_col].shift(-1, fill_value=False)

# Remplir la matrice avec les jours fériés
for _, row in df_holidays.iterrows():
    country = row["Pays"].lower()
    mask_ferie = all_date_matrix["Date"] == row["Date"]
    all_date_matrix.loc[mask_ferie, f"ferie_{country}"] = True

# # Exporter la matrice finale
# all_date_matrix.to_excel("matrice_vacances_feries_avec_regions_europe_2019_2026.xlsx", index=False)
all_date_matrix.head()

C:\Users\a.valery\AppData\Local\Temp\ipykernel_8884\3511258524.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_date_matrix[f"{region_col}_début"] = False
C:\Users\a.valery\AppData\Local\Temp\ipykernel_8884\3511258524.py:98: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_date_matrix[f"{region_col}_fin"] = False
C:\Users\a.valery\AppData\Local\Temp\ipykernel_8884\3511258524.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

,Date,ferie_fr,vac_fr_fr-ar,vac_fr_fr-ar_début,vac_fr_fr-ar_fin,vac_fr_fr-bf,vac_fr_fr-bf_début,vac_fr_fr-bf_fin,vac_fr_fr-bl,vac_fr_fr-bl_début,...,vac_es_es-nc_fin,vac_es_es-pv,vac_es_es-pv_début,vac_es_es-pv_fin,vac_es_es-ri,vac_es_es-ri_début,vac_es_es-ri_fin,vac_es_es-vc,vac_es_es-vc_début,vac_es_es-vc_fin
0,2019-01-01,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2019-01-02,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2019-01-03,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2019-01-04,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2019-01-05,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Fériés Europe avec OpenHolidays ###

In [23]:
# **Paramètres pour OpenHolidays API**
countries = ["FR", "IT", "DE", "CH", "BE", "ES"]  # France, Italie, Allemagne, Suisse, Belgique, Espagne
language = "FR"  # Langue pour les noms
years = list(range(2019, 2027))  # Années à récupérer

# Initialiser une liste pour stocker les résultats
all_holidays_openholidays = []

# Boucle pour récupérer les jours fériés pour chaque pays et chaque année
for country in countries:
    for year in years:
        # URL pour les jours fériés
        url = f"https://openholidaysapi.org/PublicHolidays?countryIsoCode={country}&languageIsoCode={language}&validFrom={year}-01-01&validTo={year}-12-31"

        # Effectuer la requête
        response = requests.get(url)

        # Vérifier la réponse
        if response.status_code == 200:
            holidays_data = response.json()

            # Ajouter les données dans la liste
            for holiday in holidays_data:
                all_holidays_openholidays.append({
                    "Date": pd.to_datetime(holiday["startDate"]),
                    "Pays": country,
                    "Nom": holiday.get("name", "Jour férié"),
                    "Type": "jour_férié"
                })
        else:
            print(f"Erreur pour {country} en {year}: {response.status_code}")

# Transformer en DataFrame
df_openholidays_holidays = pd.DataFrame(all_holidays_openholidays)

# Trier par date et pays
df_openholidays_holidays.sort_values(by=["Pays", "Date"], inplace=True)

# Réinitialiser l'index
df_openholidays_holidays.reset_index(drop=True, inplace=True)

# # Exporter si nécessaire
# df_openholidays_holidays.to_excel("jours_feries_europe_openholidays_2019_2026.xlsx", index=False)
df_openholidays_holidays.head()

,Date,Pays,Nom,Type
0,2020-01-01,BE,"[{'language': 'FR', 'text': 'Jour de l'an'}]",jour_férié
1,2020-04-12,BE,"[{'language': 'FR', 'text': 'Pâques'}]",jour_férié
2,2020-04-13,BE,"[{'language': 'FR', 'text': 'Lundi de Pâques'}]",jour_férié
3,2020-05-01,BE,"[{'language': 'FR', 'text': 'Fête du Travail'}]",jour_férié
4,2020-05-21,BE,"[{'language': 'FR', 'text': 'Ascension'}]",jour_férié


### Matrice fériés et vacances avec OpenHolidays ###

In [24]:
# **Paramètres généraux**
countries = ["FR", "IT", "DE", "CH", "BE", "ES"]  # Liste des pays
language = "FR"  # Langue pour les noms
years = list(range(2019, 2027))  # Années de 2019 à 2026

# Récupérer les jours fériés
def get_public_holidays():
    all_holidays = []
    for country in countries:
        for year in years:
            url = f"https://openholidaysapi.org/PublicHolidays?countryIsoCode={country}&languageIsoCode={language}&validFrom={year}-01-01&validTo={year}-12-31"
            response = requests.get(url)
            if response.status_code == 200:
                holidays_data = response.json()
                for holiday in holidays_data:
                    all_holidays.append({
                        "Date": pd.to_datetime(holiday["startDate"]),
                        "Pays": country,
                        "Nom": holiday.get("name", "Jour férié"),
                        "Type": "jour_férié"
                    })
            else:
                print(f"Erreur pour {country} en {year}: {response.status_code}")
    return pd.DataFrame(all_holidays)

# Récupérer les vacances scolaires
def get_school_vacations():
    all_vacations = []
    for country in countries:
        # Récupérer les subdivisions pour le pays
        url_subdivisions = f"https://openholidaysapi.org/Subdivisions?countryIsoCode={country}"
        response_subdivisions = requests.get(url_subdivisions)
        if response_subdivisions.status_code == 200:
            subdivisions = [sub.get("id", "") for sub in response_subdivisions.json()]
        else:
            print(f"Erreur lors de la récupération des subdivisions pour {country}: {response_subdivisions.status_code}")
            subdivisions = []

        # Récupérer les vacances scolaires pour chaque subdivision
        for year in years:
            for subdivision in subdivisions:
                url_vacations = f"https://openholidaysapi.org/SchoolHolidays?countryIsoCode={country}&subdivisionCode={subdivision}&languageIsoCode={language}&validFrom={year}-01-01&validTo={year}-12-31"
                response_vacations = requests.get(url_vacations)
                if response_vacations.status_code == 200:
                    vacations_data = response_vacations.json()
                    for vacation in vacations_data:
                        all_vacations.append({
                            "Date Début": pd.to_datetime(vacation["startDate"]),
                            "Date Fin": pd.to_datetime(vacation["endDate"]),
                            "Pays": country,
                            "Subdivision": subdivision,
                            "Nom": vacation.get("name", "Vacances Scolaires"),
                            "Type": "vacances"
                        })
                else:
                    print(f"Erreur pour {country} - {subdivision} en {year}: {response_vacations.status_code}")
    return pd.DataFrame(all_vacations)

# Récupérer les données
df_public_holidays = get_public_holidays()
df_school_vacations = get_school_vacations()

# Étendre les plages de vacances
expanded_vacations = []
for _, row in df_school_vacations.iterrows():
    for single_date in pd.date_range(start=row["Date Début"], end=row["Date Fin"]):
        expanded_vacations.append({
            "Date": single_date,
            "Pays": row["Pays"],
            "Subdivision": row["Subdivision"],
            "Nom": row["Nom"],
            "Type": row["Type"]
        })
df_vacations_expanded = pd.DataFrame(expanded_vacations)

# Créer la matrice centrale
date_range = pd.date_range(start="2019-01-01", end="2026-12-31", freq="D")
open_holidays_date_matrix = pd.DataFrame({"Date": date_range})

# Ajouter les colonnes pour les jours fériés
for country in countries:
    open_holidays_date_matrix[f"ferie_{country.lower()}"] = open_holidays_date_matrix["Date"].isin(
        df_public_holidays.loc[df_public_holidays["Pays"] == country, "Date"]
    )

# Ajouter les colonnes pour les vacances scolaires
for _, row in df_vacations_expanded.iterrows():
    country = row["Pays"].lower()
    subdivision = row["Subdivision"].lower()
    col_name = f"vac_{country}_{subdivision}"
    if col_name not in open_holidays_date_matrix.columns:
        open_holidays_date_matrix[col_name] = False
    open_holidays_date_matrix.loc[open_holidays_date_matrix["Date"] == row["Date"], col_name] = True

# # Exporter les résultats
# open_holidays_date_matrix.to_excel("matrice_vacances_feries_europe_openholidays_2019_2026.xlsx", index=False)
open_holidays_date_matrix.head()

,Date,ferie_fr,ferie_it,ferie_de,ferie_ch,ferie_be,ferie_es,vac_fr_,vac_it_,vac_de_,vac_ch_,vac_be_,vac_es_
0,2019-01-01,False,False,False,False,False,False,False,False,True,False,False,False
1,2019-01-02,False,False,False,False,False,False,False,False,True,False,False,False
2,2019-01-03,False,False,False,False,False,False,False,False,True,False,False,False
3,2019-01-04,False,False,False,False,False,False,False,False,True,False,False,False
4,2019-01-05,False,False,False,False,False,False,False,False,False,False,False,False
